## A. Xử lý Dữ liệu

In [3]:
# import thư viện
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import ast
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
import pickle 
import warnings

In [7]:
df2=pd.read_csv('reviewdata.csv')
df2=df2[['userId','productId','Rating']]
df2.head(5)

,userId,productId,Rating
0,709310,10001012,3
1,10701688,10001012,5
2,11763074,10001012,5
3,9909549,10001012,5
4,1827148,10001012,5


In [5]:
data2=df2.rename(columns={"customer_id": "userId", "product_id": "productId","rating": "Rating"})
data2.head()

,userId,productId,Rating
0,709310,10001012,3
1,10701688,10001012,5
2,11763074,10001012,5
3,9909549,10001012,5
4,1827148,10001012,5


In [142]:
electronics_groupby_users_Ratings = data2.groupby('userId')['Rating']
electronics_groupby_users_Ratings = pd.DataFrame(electronics_groupby_users_Ratings.count())

In [195]:
user_list_min50_ratings = electronics_groupby_users_Ratings[electronics_groupby_users_Ratings['Rating'] >= 2].index
electronics =  data2[data2['userId'].isin(user_list_min50_ratings)]

In [196]:
len(user_list_min50_ratings)

60881

In [197]:
print('The total number of rows :', electronics.shape[0])
print('The total number of columns :', electronics.shape[1])

The total number of rows : 173489
The total number of columns : 4


In [198]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(electronics, test_size =.20, random_state=10)


print(f'Training set has {train_data.shape[0]} rows and {train_data.shape[1]} columns')
print(f'Testing set has {test_data.shape[0]} rows and {test_data.shape[1]} columns')

Training set has 138791 rows and 4 columns
Testing set has 34698 rows and 4 columns


In [15]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_color_codes()
sns.set(style="whitegrid")
%matplotlib inline
from scipy.stats import zscore
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from math import sqrt

Popularity Based Recommender Model: Popularity based recommendation system uses the items that are in trend right now. It ranks products based on its popularity i.e. the rating count. If a product is highly rated then it is most likely to be ranked higher and hence will be recommended. As it is based on the products popularity, this can not be personalized and hence same set of products will be recommended for all the users.

In [200]:
train_data, test_data = train_test_split(electronics, test_size =.30, random_state=10)

print(f'Training set has {train_data.shape[0]} rows and {train_data.shape[1]} columns')
print(f'Testing set has {test_data.shape[0]} rows and {test_data.shape[1]} columns')

Training set has 121442 rows and 4 columns
Testing set has 52047 rows and 4 columns


In [210]:
class popularity_based_recommender_model():
    def __init__(self, train_data, test_data, user_id, item_id):
        self.train_data = train_data
        self.test_data = test_data
        self.user_id = user_id
        self.item_id = item_id
        self.popularity_recommendations = None
        
    #Create the popularity based recommender system model
    def fit(self):
        #Get a count of user_ids for each unique product as recommendation score
        train_data_grouped = train_data.groupby([self.item_id]).agg({self.user_id: 'count'}).reset_index()
        train_data_grouped.rename(columns = {'userId': 'score'},inplace=True)
    
        #Sort the products based upon recommendation score
        train_data_sort = train_data_grouped.sort_values(['score', self.item_id], ascending = [0,1])
    
        #Generate a recommendation rank based upon score
        train_data_sort['Rank'] = train_data_sort['score'].rank(ascending=0, method='first')
        
        #Get the top 10 recommendations
        self.popularity_recommendations = train_data_sort.head(20)

    #Use the popularity based recommender system model to make recommendations
    def recommend(self, user_id, n=5):    
        user_recommendations = self.popularity_recommendations
        
        #Filter products that are not rated by the user
        products_already_rated_by_user = self.train_data[self.train_data[self.user_id] == user_id][self.item_id]        
        user_recommendations = user_recommendations[~user_recommendations[self.item_id].isin(products_already_rated_by_user)]
        
        #Add user_id column for which the recommendations are being generated
        user_recommendations['user_id'] = user_id
    
        #Bring user_id column to the front
        cols = user_recommendations.columns.tolist()
        cols = cols[-1:] + cols[:-1]
        user_recommendations = user_recommendations[cols].head(n)     
       
        return user_recommendations
    
    
    def predict_evaluate(self):        
        ratings = pd.DataFrame(self.train_data.groupby(self.item_id)['Rating'].mean())
        
        pred_ratings = [];            
        for data in self.test_data.values:
            if(data[1] in (ratings.index)):
                pred_ratings.append(ratings.loc[data[1]])
            else:
                pred_ratings.append(0)
        
        mse = mean_squared_error(self.test_data['Rating'], pred_ratings)
        rmse = sqrt(mse)
        return rmse
        

In [211]:
pr = popularity_based_recommender_model(train_data=train_data, test_data=test_data, user_id='userId', item_id='productId')

In [212]:
pr.fit()

In [205]:
pr.predict_evaluate()

4.671668391605663

In [213]:
result_pop_user1 = pr.recommend('10701688')
result_pop_user1

,user_id,productId,score,Rank
30,10701688,299461,1637,1.0
2869,10701688,54017350,1000,2.0
155,10701688,487511,918,3.0
64,10701688,405243,822,4.0
2315,10701688,43107185,798,5.0


In [226]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import scipy.sparse as sparse
from sklearn.metrics.pairwise import cosine_similarity
import operator
%matplotlib inline


sparse_matrix = sparse.coo_matrix((df2['Rating'].values,(df2['userId'].values,df2['productId'].values)))
# Convert the sparse matrix to compressed sparse row (CSR) format
csr_matrix = sparse_matrix.tocsr()
k = 5
model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=k)
model.fit(csr_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [20]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import scipy.sparse as sparse
from sklearn.metrics.pairwise import cosine_similarity
import operator
%matplotlib inline


sparse_matrix = sparse.coo_matrix((df2['Rating'].values,(df2['userId'].values,df2['productId'].values)))
# Convert the sparse matrix to compressed sparse row (CSR) format
csr_matrix = sparse_matrix.tocsr()
k = 5
model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=k)
model.fit(csr_matrix)
user_id = 709310
distances, indices = model.kneighbors(csr_matrix.getrow(user_id), n_neighbors=k+1)

# Find the indices of the k most similar users to user_id
distances, indices = model.kneighbors(csr_matrix.getrow(user_id), n_neighbors=k+1)

# Get the indices of the products that the k most similar users have rated
product_indices = csr_matrix[indices.flatten()[1:], :].nonzero()[1]

# Get the unique product IDs from the product_indices
product_ids = np.unique(product_indices)

print(f"The k most similar users to user {user_id} have rated {len(product_ids)} unique products.")
print("Product IDs:")
print(product_ids)


The k most similar users to user 709310 have rated 3 unique products.
Product IDs:
[ 2927891 10001012 14868418]
